<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('data/caracteristiques-2019.csv', sep = ';')
df.head()
len(df)

58840

In [3]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from urllib.request import urlopen as uReq
import matplotlib.pyplot  as plt
import seaborn as sns

In [4]:
url = "https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/#_"

In [5]:
# function to get page soup from html page
def get_page_soup(url):
    # opening connection
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    #html parser
    return soup(page_html, "html.parser")


In [6]:
page  = get_page_soup(url)
articles = page.findAll("article")
liens = [[],[],[],[]]
names = ['caracteristiques', 'lieux', 'vehicules','usagers']
for a in articles:
    if a.div.h4 !=None:
        name = a.div.h4.text.split('-')[0].split('_')[0]
    for i, n in enumerate(names):
        if name == n:
            lien = a.footer.find("div", {"resource-card-actions btn-toolbar"}).findAll("a")[1]['href']
            liens[i].append(lien)



In [8]:
# to create data base on your laptop, call function create_data_base,
# do not forget to create empty file with name accidents.sqlite3 in the same directory your notebook is.
def create_data_base():
#Import libraries
    import pandas as pd, csv, sqlite3

    # Create sqlite database and cursor
    conn = sqlite3.connect('accidents.sqlite3')
    c = conn.cursor()

    #c.execute("""DROP TABLE IF EXISTS caracteristiques, lieux, vehicules, usagers;""")
    c.execute("""CREATE TABLE IF NOT EXISTS caracteristiques (Num_Acc INTEGER, jour INTEGER, mois INTEGER, an INTEGER, hrmn TEXT, lum INTEGER, dep TEXT, com TEXT, agg INTEGER, int INTEGER, atm INTEGER, col INTEGER, adr TEXT, gps TEXT, lat TEXT, long TEXT)""")
    c.execute("""CREATE TABLE IF NOT EXISTS lieux (Num_Acc INTEGER, catr INTEGER, voie REAL, v1 REAL, v2 TEXT, circ REAL, nbv REAL, pr REAL, pr1 REAL, vosp REAL, prof REAL, plan REAL, lartpc REAL, larrout REAL, surf REAL, infra REAL, situ REAL, env1 REAL, vma INTEGER)""")
    c.execute("""CREATE TABLE IF NOT EXISTS vehicules (Num_Acc INTEGER, id_vehicule TEXT, senc REAL, catv INTEGER, occutc INTEGER, obs REAL, obsm REAL, choc REAL, manv REAL, num_veh TEXT, motor INTEGER)""")
    c.execute("""CREATE TABLE IF NOT EXISTS usagers (Num_Acc INTEGER, id_vehicule TEXT, place REAL, catu INTEGER, grav INTEGER, sexe INTEGER, trajet REAL, secu REAL, secu1 REAL, secu2 REAL, secu3 REAL, locp REAL, actp REAL, etatp REAL, an_nais REAL, num_veh TEXT)""")
    conn.commit()


    for i,lien in enumerate(liens): #names 0,1,2,3
        for j,l in enumerate(lien): #years 0-15
            print("fichier {} - {}".format(names[i],2019-j))
            #print(l)
            try:
                df = pd.read_csv(l, sep=';', low_memory=False) #, encoding = 'ISO-8859-1', error_bad_lines=False, quotechar='"'

                if (len(df.columns)<2):
                    df = pd.read_csv(l, sep = ',', low_memory=False)
            except:
                try:
                    print('exeption1')
                    df = pd.read_csv(l, sep = ',', encoding = 'ISO-8859-1', engine = 'python')

                except:
                    print('exeption2')
                    df = pd.read_csv(l, sep = '\t', encoding = 'ISO-8859-1', engine = 'python')


            df.to_sql(names[i], conn, if_exists='append', index=False)


In [10]:
import pandas as pd, csv, sqlite3
conn = sqlite3.connect('accidents.sqlite3')
usagers = pd.read_sql_query(f"SELECT * FROM {names[3]}", conn)
usagers 

,Num_Acc,id_vehicule,place,catu,grav,sexe,trajet,secu,secu1,secu2,secu3,locp,actp,etatp,an_nais,num_veh
0,201900000001,138 306 524,2.0,2,4,2,0.0,NaN,1.0,0.0,-1.0,-1.0,-1.0,-1.0,2002.0,B01
1,201900000001,138 306 524,1.0,1,4,2,5.0,NaN,1.0,0.0,-1.0,-1.0,-1.0,-1.0,1993.0,B01
2,201900000001,138 306 525,1.0,1,1,1,0.0,NaN,1.0,0.0,-1.0,-1.0,-1.0,-1.0,1959.0,A01
3,201900000002,138 306 523,1.0,1,4,2,0.0,NaN,1.0,0.0,-1.0,-1.0,-1.0,-1.0,1994.0,A01
4,201900000003,138 306 520,1.0,1,1,1,0.0,NaN,1.0,0.0,-1.0,-1.0,0.0,-1.0,1996.0,A01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275167,200500087953,None,1.0,1,1,1,0.0,13.0,NaN,NaN,NaN,0.0,0.0,0.0,1972.0,B01
2275168,200500087953,None,1.0,1,1,1,0.0,13.0,NaN,NaN,NaN,0.0,0.0,0.0,1965.0,C01
2275169,200500087953,None,1.0,1,4,1,5.0,23.0,NaN,NaN,NaN,0.0,0.0,0.0,1990.0,A01
2275170,200500087954,None,1.0,1,4,1,0.0,13.0,NaN,NaN,NaN,0.0,0.0,0.0,1951.0,A01


In [42]:
usagers.isnull().sum()

Num_Acc              0
id_vehicule    2142195
place           123128
catu                 0
grav                 0
sexe                 0
trajet             494
secu            189514
secu1          2142195
secu2          2142195
secu3          2142195
locp             56361
actp             56462
etatp            56418
an_nais           2418
num_veh              0
dtype: int64

In [43]:
vehicules = pd.read_sql_query(f"SELECT * FROM {names[2]}", conn)
vehicules 

,Num_Acc,id_vehicule,senc,catv,occutc,obs,obsm,choc,manv,num_veh,motor
0,201900000001,138 306 524,2.0,7,NaN,0.0,2.0,5.0,23.0,B01,1.0
1,201900000001,138 306 525,2.0,17,NaN,1.0,0.0,3.0,11.0,A01,1.0
2,201900000002,138 306 523,1.0,7,NaN,4.0,0.0,1.0,0.0,A01,1.0
3,201900000003,138 306 520,1.0,7,NaN,0.0,2.0,1.0,2.0,A01,1.0
4,201900000003,138 306 521,1.0,7,NaN,1.0,0.0,4.0,2.0,B01,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1736516,200500087952,None,0.0,7,0.0,0.0,2.0,3.0,1.0,A01,NaN
1736517,200500087953,None,0.0,7,0.0,0.0,2.0,3.0,19.0,B01,NaN
1736518,200500087953,None,0.0,2,0.0,0.0,2.0,1.0,17.0,A01,NaN
1736519,200500087953,None,0.0,7,0.0,0.0,2.0,1.0,1.0,C01,NaN
